In [7]:


from PIL import Image
import os


In [8]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import matplotlib.pyplot as plt



In [9]:
#defining generator model
class Generator(nn.Module):
  def __init__(self):
    super(Generator,self).__init__()
    #calling nn module parameteralized constructor to initialize its internal state
    self.model=nn.Sequential(
        #creating 2d convulation with 1 input channel and 64 output channels(64 filters)
        #1st convulation layer
        nn.Conv2d(1,64,kernel_size=3,stride=1,padding=1),
        nn.LeakyReLU(0.2),
        nn.BatchNorm2d(64),
        #2nd convulation layer
        nn.Conv2d(64,128,kernel_size=3,stride=1,padding=1),
        nn.LeakyReLU(0.2),
        nn.BatchNorm2d(128),
        #3rd convulation layer
        nn.Conv2d(128,512,kernel_size=3,stride=1,padding=1),
        nn.LeakyReLU(0.2),
        nn.BatchNorm2d(512),
        #4th convulation layer
        nn.ConvTranspose2d(512, 256, kernel_size=3, stride=2, padding=1, output_padding=1),
        nn.LeakyReLU(0.2),
        nn.BatchNorm2d(256),
        #5th
        nn.ConvTranspose2d(256, 128, kernel_size=3, stride=2, padding=1, output_padding=1),
        nn.LeakyReLU(0.2),
        nn.BatchNorm2d(128),
        #6th convulation layers
        nn.ConvTranspose2d(128, 64, kernel_size=3, stride=2, padding=1, output_padding=1),
        nn.LeakyReLU(0.2),
        nn.BatchNorm2d(64),
        #7th convulation layer
        nn.Conv2d(64, 3, kernel_size=3, stride=1, padding=1),
        nn.Tanh()
    )
  #this generator will produce 2048*2048 3RGBmatrix
  def forward(self,x):
    return self.model(x)

In [17]:
class Discriminator(nn.Module):
  def __init__(self):
    super(Discriminator,self).__init__()
    self.model=nn.Sequential(
      #1 conv layer
      nn.Conv2d(3,64,kernel_size=3,stride=2,padding=1),#2048 to 1024
      nn.LeakyReLU(0.2),
      nn.Dropout(0.25),
      #randomly drop 25% neurons to avoid overfitting
      nn.Conv2d(64,128,kernel_size=3,stride=2,padding=1),#1024 to 512
      nn.BatchNorm2d(128),
      nn.Dropout(0.25),

      nn.Conv2d(128, 256, kernel_size=3, stride=2, padding=1),#512 to 256
      nn.LeakyReLU(0.2),
       nn.BatchNorm2d(256),
      nn.Dropout(0.25),

     nn.Conv2d(256, 512, kernel_size=3, stride=1, padding=1),#256 to 256
     nn.LeakyReLU(0.2),
     nn.BatchNorm2d(512),
     nn.Dropout(0.25),

     nn.Flatten(),
     nn.Linear(524288,1),
     nn.Sigmoid()

      #flattening to convert the multidimensional tensor
      #to single dimensional tensor so that it can be taken as input for classification task done by neural descriminator



    )
  def forward(self,x):
    return self.model(x)

In [11]:
#defining loss functions and optimizers
criterion = nn.BCELoss()

def get_optimizer(model, lr=0.0002, beta1=0.5):
    return optim.Adam(model.parameters(), lr=lr, betas=(beta1, 0.999))
#loss function is Binary cross entropy loss and optimizer is Adam
def load_image(image_path):
    image = Image.open(image_path)
    transform = transforms.ToTensor()  # Converts image to PyTorch tensor
    return transform(image)

def load_data(sar_dir, optical_dir):
    sar_images = []
    optical_images = []


    sar_files = sorted(os.listdir(sar_dir))
    optical_files = sorted(os.listdir(optical_dir))

    for filename1, filename2 in zip(sar_files, optical_files):
        sar_image = load_image(os.path.join(sar_dir, filename1))
        optical_image = load_image(os.path.join(optical_dir, filename2))
        sar_images.append(sar_image[:, 0:1])  # Keep only the first channel (Grayscale)
        optical_images.append(optical_image)

    return torch.stack(sar_images), torch.stack(optical_images)  # Stack to form a batch

def normalize(tensor):
    return (tensor - 0.5) * 2

def normalize_images(sar_dir, optical_dir):
    sar_images_tensor, optical_images_tensor = load_data(sar_dir, optical_dir)

    normalized_sar_images = normalize(sar_images_tensor)
    normalized_optical_images = normalize(optical_images_tensor)

    return normalized_sar_images, normalized_optical_images
from sklearn.model_selection import train_test_split
def train_test_split_data(sar_images, optical_images, test_size=0.2):
    # Flatten the list of images into 1D array for splitting
    indices = list(range(len(sar_images)))
    train_idx, test_idx = train_test_split(indices, test_size=test_size, random_state=42)

    sar_train = sar_images[train_idx]
    optical_train = optical_images[train_idx]

    sar_test = sar_images[test_idx]
    optical_test = optical_images[test_idx]

    return sar_train, optical_train, sar_test, optical_test



In [12]:
sar_dir = "C:/Users/sujal/PycharmProjects/scientificProject/sar_image"
optical_dir = r"C:/Users/sujal/PycharmProjects/scientificProject/optical_image"
normalized_sar_images, normalized_optical_images = normalize_images(sar_dir, optical_dir)

In [13]:
normalized_optical_images,normalized_sar_images


(tensor([[[[ 0.1765, -0.0510, -0.3020,  ..., -0.3255, -0.3961, -0.4196],
           [ 0.1529,  0.1686, -0.1137,  ..., -0.1765, -0.2941, -0.4353],
           [ 0.1059,  0.0902, -0.0745,  ..., -0.3569, -0.2392, -0.2941],
           ...,
           [-0.3882, -0.3412, -0.4510,  ...,  0.8118,  0.5686,  1.0000],
           [-0.3961,  0.2784, -0.3961,  ...,  0.5216,  0.3804,  0.4745],
           [ 0.2784,  0.8118, -0.0196,  ..., -0.0196, -0.0196, -0.0039]],
 
          [[ 0.0510, -0.0275, -0.2235,  ..., -0.3412, -0.3882, -0.4824],
           [ 0.0588,  0.0745, -0.0510,  ..., -0.3412, -0.4196, -0.4824],
           [-0.0275, -0.0510, -0.0980,  ..., -0.2235, -0.2706, -0.3882],
           ...,
           [-0.3412, -0.2000, -0.3412,  ...,  0.7804,  0.2627,  1.0000],
           [-0.1451,  0.5059, -0.2863,  ...,  0.4275,  0.2078,  0.3020],
           [ 0.5529,  0.8588, -0.0510,  ..., -0.0588, -0.0824, -0.0667]],
 
          [[-0.1608, -0.2863, -0.5059,  ..., -0.5216, -0.5765, -0.5922],
           [-

In [14]:
def train_gan(generator, discriminator, gan, sar_train, optical_train, sar_test, optical_test, epochs=10000, batch_size=32):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    generator.to(device)
    discriminator.to(device)
    gan.to(device)

    valid = torch.ones((batch_size, 1), device=device)
    fake = torch.zeros((batch_size, 1), device=device)

    for epoch in range(epochs):
        idx = torch.randint(0, sar_train.size(0), (batch_size,))
        sar_batch = sar_train[idx].to(device)
        optical_batch = optical_train[idx].to(device)

        # Training discriminator
        discriminator.zero_grad()
        real_loss = criterion(discriminator(optical_batch), valid)
        fake_images = generator(sar_batch).detach()
        fake_loss = criterion(discriminator(fake_images), fake)
        d_loss = (real_loss + fake_loss) / 2
        d_loss.backward()
        optimizer_d.step()
        
        #training the generator
        generator.zero_grad()
        g_loss = criterion(discriminator(generator(sar_batch)), valid)
        g_loss.backward()
        optimizer_g.step()

        if epoch % 100 == 0:
            print(f"{epoch} [D loss: {d_loss.item()}] [G loss: {g_loss.item()}]")
            sample_images(generator, sar_batch, epoch, sar_test)
            
def sample_images(generator, sar_images, epoch, sar_test):
    generator.eval()
    with torch.no_grad():
        generated_images = generator(sar_images).cpu().numpy()
        test_images = generator(sar_test).cpu().numpy()

    r, c = 2, 3
    fig, axs = plt.subplots(r, c)
    for i in range(c):
        axs[0, i].imshow(sar_images[i].cpu().squeeze(0).squeeze(0), cmap='gray')
        axs[1, i].imshow((generated_images[i].transpose(1, 2, 0) + 1) / 2)  # Rescale [-1, 1] to [0, 1]
        axs[1, c + i].imshow((test_images[i].transpose(1, 2, 0) + 1) / 2)  # Rescale [-1, 1] to [0, 1]
    plt.show()
        

        

        

    
    




In [15]:
class GAN(nn.Module):
    def __init__(self, generator, discriminator):
        super(GAN, self).__init__()
        self.generator = generator
        self.discriminator = discriminator

    def forward(self, z):
        # Generate images from the generator
        generated_images = self.generator(z)
        # Classify the generated images with the discriminator
        validity = self.discriminator(generated_images)
        return validity

In [18]:
generator = Generator()
discriminator = Discriminator()
gan = GAN(generator, discriminator)

optimizer_g = get_optimizer(generator)
optimizer_d = get_optimizer(discriminator)
sar_train, optical_train, sar_test, optical_test = train_test_split_data(normalized_sar_images,normalized_optical_images, test_size=0.2)


train_gan(generator, discriminator, gan, sar_train, optical_train, sar_test, optical_test, epochs=10000, batch_size=32)


RuntimeError: mat1 and mat2 shapes cannot be multiplied (32x131072 and 524288x1)